## 2: Improving our Math

In the previous notebook we played with some basic parameters. Let's tighten up on our math outputs here.

As before, we'll start with the imports

In [5]:
import os, openai, json

Now we need to set the API key and (if applicable) the organization you belong to if that's set up. You can manually put them in or bring them in from the environement.

In [2]:
openai.api_key = os.environ['OPENAI_KEY']

In [3]:
try:
    openai.organization = os.environ['OPENAI_ORG']
    print('Organization Added')
except:
    print('No Organization')

Organization Added


Run this if you want to charge your personal instead of org

In [4]:
#openai.organization=None

To call openai, we need to do an openai.Completion.create() with a bunch of parameters. To make this easier, we'll create a wrapper function that (1) has some default parameters as well as (2) saves each request/response to file for us.

Let's start by creating a folder to drop prompts

In [6]:
os.makedirs('prompts', exist_ok=True)

now we'll actually make the wrapper function that can take args as well as has defaults

In [24]:
def query(prompt, full=False, **kwargs):
    modelKwargs = {
        'temperature':0,
        'engine':'davinci',
        'max_tokens':10,
        'stop':['\n']
    }
    
    for kwarg in kwargs:
        modelKwargs[kwarg] = kwargs[kwarg]
    
    completion = openai.Completion.create(prompt=prompt, **modelKwargs)
    
    with open('prompts/{}.json'.format(completion['id']), 'w') as fh:
        json.dump(completion, fh, indent=4)
    
    if full:
        return completion
    if 'n' in modelKwargs:
        return [x['text'] for x in completion['choices']]
    return completion['choices'][0]['text']

let's make sure it works!

In [27]:
query('1+1=')

'2.'

K, so for an example, we'll see if we can have our model spit out the written form of each addition problem rather than the numeral. E.g., instead of 1+1=2, we want 1+1=two.

We can use string formatting to set up the prompt for us to test different inputs. We'll give it a single example and see if that's enough for it to pattern match.

In [25]:
prompt = """Question: 1+1
Answer:two

Question: {}
Answer:"""

In [29]:
print(prompt.format('FOO'))

Question: 1+1
Answer:two

Question: FOO
Answer:


In [26]:
query(prompt.format('2+2'))

'four'

In [28]:
query(prompt.format('5+4'))

'nine'

Well, how's it do when we get above single digit numbers?

In [30]:
query(prompt.format('5+10'))

'15'

In [31]:
query(prompt.format('15+10'))

'25'

We can expand the prompt so it covers larger numbers too

In [32]:
prompt = """Question: 1+1
Answer: two

Question: 12 + 52
Answer: sixty-four

Question: {}
Answer:"""

In [33]:
query(prompt.format('15+10'))

' twenty-five'

Horray! that might have done it... but does it hold up?

In [34]:
query(prompt.format('115+10'))

' 125'

In [35]:
query(prompt.format('65+50'))

' 115'

In [36]:
query(prompt.format('65+23'))

' 88'

In [37]:
query(prompt.format('65 + 23'))

' 88'

We can see if we can give it hints to do it correctly

In [38]:
prompt = """Task: Spell out the sum of each math problem.

Question: 1+1
Answer: two

Question: 12 + 52
Answer: sixty-four

Question: {}
Answer:"""

In [39]:
query(prompt.format('65 + 23'))

' eighty-eight'

In [40]:
query(prompt.format('165 + 80'))

' 255'

But it doesn't always work, so let's try just giving more examples instead

In [56]:
prompt = """Task: Spell out the sum of each math problem.

Question: 40 + 50
Answer: Ninety

Question: 1+1
Answer: Two

Question: 50 + 80
Answer: One hundred and twenty

Question: 12 + 52
Answer: Sixty Four

Question: {}
Answer:"""

In [57]:
query(prompt.format('65 + 23'))

' 88'

Man, that still didn't work. Let's try really emphasizing that it's spelled out

In [58]:
prompt = """Task: Spell out the sum of each math problem.

Question: 40 + 50
Spelled Out Answer: Ninety

Question: 1+1
Spelled Out Answer: Two

Question: 50 + 80
Spelled Out Answer: One hundred and twenty

Question: 12 + 52
Spelled Out Answer: Sixty Four

Question: {}
Spelled Out Answer:"""

In [59]:
query(prompt.format('65 + 23'))

' Eighty Eight'

In [60]:
query(prompt.format('165 + 23'))

' One hundred and ninety eight'

Well, GPT's bad at math so that last one isn't necessarily it's fault. However, we can improve it by having it do the math first and then spelling it out.

In [65]:
prompt = """Task: Spell out the sum of each math problem.

Question: 40 + 50
Answer: 90
Spelled Out: Ninety

Question: 1+1
Answer: 2
Spelled Out: Two

Question: 50 + 80
Answer: 130
Spelled Out: One hundred and twenty

Question: 12 + 52
Answer: 64
Spelled Out: Sixty Four

Question: {}
Answer:"""

In [66]:
query(prompt.format('165 + 23'))

' 188'

Whoops, it didn't give us the full answer because we had it stop on '\n'

In [67]:
r = query(prompt.format('165 + 23'), stop=['\n\n'])

In [68]:
print(r)

 188
Answer: One hundred and eighty eight



Horray! So we can engineer prompts to do more and more. How's it do with 4 digits?

In [70]:
r = query(prompt.format('565 + 700'), stop=['\n\n'], max_tokens=30)
print(r)

 1265
Spelled Out: One thousand two hundred and sixty five


In [71]:
r = query(prompt.format('1565 + 700'), stop=['\n\n'], max_tokens=30)
print(r)

 2465
Spelled Out: Twenty Four Hundred and Sixty Five


Well, Math's hard. I'll leave this with a problem for you: Is there a way to improve its ability to do 4 digit arithmetic?

In [73]:
prompt = """
YOUR PROMPT HERE
"""